In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
my_url = 'https://www.newegg.com/p/pl?d=shoes'

In [0]:
result = requests.get(my_url)

In [0]:
#confirm if the request was sucessful, an outputof 200 means success
print(result.status_code)

200


In [0]:
#extrainformation on the webpage
print(result.headers)

{'Content-Type': 'text/html', 'NEL': '{"report_to": "default", "max_age": 2592000}`);', 'Report-To': '{"group":"default","max-age":10886400,"include_subdomains":"true","endpoints":[{"url":"https://pf.newegg.com/csp"}]};', 'Content-Security-Policy': "frame-ancestors 'self';, upgrade-insecure-requests", 'X-Frame-Options': 'SAMEORIGIN', 'Content-Encoding': 'gzip', 'Cache-Control': 'private, max-age=0', 'Date': 'Fri, 03 Jan 2020 19:13:47 GMT', 'Content-Length': '3511', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Set-Cookie': 'NV_NVTCTIMESTAMP=1578078828; domain=.newegg.com; expires=Mon, 02-Jan-23 19:13:47 GMT; path=/;secure, NSC_mc-xxx.ofxfhh.dpn-vsmibti=5ccba3d8e5298caab4a589a9faf558cf92718980218706d442222e96b59e237f32fc8ef3;expires=Fri, 03-Jan-2020 19:42:25 GMT;path=/;secure', 'Strict-Transport-Security': 'max-age=31536000'}


In [0]:
#return the source of the page, obtains the content of the webpage
src = result.content
print(src)

b'\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en">\n<head>\n<title>Are you a human?</title>\n<meta http-equiv="X-UA-Compatible" content="IE=EDGE" /><meta charset="UTF-8" />\n<link rel="shortcut icon" type="image/x-icon" href="//c1.neweggimages.com/WebResource/Themes/2005/Nest/Newegg.ico">\n<link rel="stylesheet" type="text/css" href="https://fonts.googleapis.com/css?family=Open+Sans:400,300,300italic,400italic,700,700italic|Open+Sans+Condensed:300,300italic,700">\n<style type="text/css">\np{font-size:16px;color:#4d4d4d;padding:0;margin:0 0 5px}a img,a:hover img,a:visited img{border:0}.button-primary,.button-primary:focus,.button-primary:link,.button-primary:visited{font-family:\'Open Sans Condensed\',\'Arial Narrow\',\'Helvetica Narrow\',arial,helvetica,sans-serif;letter-spacing:1px;font-size:14px;font-weight:700;font-stretch:condensed;text-align:center;text-decoration:none;cursor:pointer;border-radius:4px;border:2px solid #E68626;display:inline-block;padding:9p

In [0]:
#parse and process the source with beautiful soup
soup = BeautifulSoup(src, 'lxml')

In [0]:
#use soup to get specific items from the webpage
links = soup.find_all('a')
print(links)
print('\n')

[<a href="https://www.newegg.com/" style="border: none;">
<img alt="" src="//images10.newegg.com/WebResource/Themes/2005/Nest/logo_print.png" width="216"/>
</a>, <a href="#" onclick="javascript:newegg_inhouse_feedback2 &amp;&amp; newegg_inhouse_feedback2.show();" style="cursor: pointer;text-decoration: underline; color: #000;" title="Request Access to Newegg API">submit a request</a>, <a href="#" id="newegg_footer_feedback" onclick="javascript:newegg_inhouse_feedback &amp;&amp; newegg_inhouse_feedback.show();" style="cursor: pointer;text-decoration: underline; color: #000;" title="feedback">Let us know your feedback</a>]




In [0]:
#grabs each product
list_wrap = soup.findAll('div', {'class' : 'item-container'})
wanted = list_wrap[4:]

Use find() to find specific tags in an item container like this;

>item_container.find('tag', {'class/id' : 'class-name'})

In [0]:
name=[]
branding = []
original_price = []
current_price = []
ship = []
for item in wanted:
    item_name = item.find('a', class_ = 'item-title').get_text()
    name.append(item_name)
    
    item_descr = item_name.split()
    brand = item_descr[0]
    branding.append(brand)
    
    item_price_old = item.find('li',  class_="price-was").get_text().strip()
    original_price.append(item_price_old[0:7])
    
    current_prices = item.find('li', {'class' : "price-current"})
    item_price_current = '$' + (current_prices.strong).get_text() + (current_prices.sup).get_text()
    current_price.append(item_price_current if item_price_current else N/A)
    
    shipping = item.find('li',  {'class' :"price-ship"}).get_text().strip()
    ship.append(shipping)
    

print(name)
print('\n')
print(branding)
print('\n')
print(original_price)
print('\n')
print(current_price)
print('\n')
print('shipping: ', ship) 

[]


[]


[]


[]


shipping:  []


In [0]:
len(wanted)

0

In [0]:
import pandas as pd
Newegg_shoes = pd.DataFrame({'Shoe_name': name,
'Brand': branding,
'Original_price': original_price,
'Current_price': current_price,
'Shipping': ship
})
print(Newegg_shoes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
Shoe_name         0 non-null float64
Brand             0 non-null float64
Original_price    0 non-null float64
Current_price     0 non-null float64
Shipping          0 non-null float64
dtypes: float64(5)
memory usage: 124.0 bytes
None


In [0]:
print(len(name))
print(len(branding))
print(len(original_price))
print(len(current_price))
print(len(ship))
#https://www.dataquest.io/blog/web-scraping-beautifulsoup/

36
36
36
36
36


Now to scrape data from multiple pages

In [0]:
import csv
import requests
from bs4 import BeautifulSoup    
import datetime

end_page_num = 50

filename = "new_egg_shoes.csv"
with open(filename, "w+") as f:

    writer = csv.writer(f)
    writer.writerow(["Name","Normal_price","Discount_price", 'Shipping', 'Data'])
    i = 1
    while i <= end_page_num:

        r = requests.get("https://www.newegg.com/p/pl?d=shoes&Page={}".format(i))

        soup = BeautifulSoup(r.text, "html.parser")
        list_wrap = soup.findAll('div', {'class' : 'item-container'})
        x = list_wrap[4:]
        
        for item in x:
            Name = item.find('a', class_ = 'item-title').get_text()
    
            item_price_old = item.find('li',  class_="price-was").get_text().strip()
            Normal_price = item_price_old[0:7]
    
            current_prices = item.find('li', {'class' : "price-current"})
            Discount_price = '$' + (current_prices.strong).get_text() + (current_prices.sup).get_text()
    
            Shipping = item.find('li',  {'class' :"price-ship"}).get_text().strip()
            writer.writerow([Name, Normal_price, Discount_price, Shipping, datetime.datetime.now()])
        i += 1

In [0]:
new_egg = pd.read_csv('new_egg_shoes.csv', encoding='latin-1')

In [0]:
new_egg.head()

,Name,Normal_price,Discount_price,Shipping,Data


In [0]:
new_egg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
Name              0 non-null object
Normal_price      0 non-null object
Discount_price    0 non-null object
Shipping          0 non-null object
Data              0 non-null object
dtypes: object(5)
memory usage: 0.0+ bytes


In [0]:
new_egg['Normal_price'].head(20)

0          NaN
1      $130.00
2          NaN
3          NaN
4          NaN
5          NaN
6      $120.00
7     $90.00\r
8     $95.98\r
9      $199.00
10     $199.00
11     $117.49
12         NaN
13    $95.98\r
14    $87.50\r
15    $75.00\r
16    $75.00\r
17    $38.00\r
18         NaN
19         NaN
Name: Normal_price, dtype: object